In [37]:
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

#Embedding text
inputs = processor(text=["a photo of a cat", "a photo of a dog", "two cats laying in a sofa"], return_tensors="pt", padding=True).to(device)
text_emb = model.get_text_features(**inputs)
text_emb = text_emb.squeeze(0).cpu().detach().numpy()
print(text_emb.shape)

#Embbeding images
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(images=image, return_tensors="pt", padding=True).to(device)
image_emb = model.get_image_features(**inputs)
image_emb = image_emb.squeeze(0).cpu().detach().numpy()
print(image_emb.shape)



`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


(3, 768)
(768,)


In [43]:
#Save embeddings in Pinecone
import pinecone

pinecone.init(api_key="04d65f27-278f-40f0-9cfb-c907b84115a7", environment="us-east4-gcp")
index_name="text-embeddings1"

if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension = 768, metric = "cosine")

index = pinecone.Index(index_name)

upsert_response = index.upsert(
    vectors=[
        ("vec1", text_emb[0].tolist()),  
        ("vec2", text_emb[1].tolist()), 
        ("vec3", text_emb[2].tolist())
    ],
    namespace="example-namespace"
)

print(index.describe_index_stats())

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'example-namespace': {'vector_count': 3}},
 'total_vector_count': 3}


In [49]:
#Saving embeddings in different spaces (used to divide different elements)

upsert_response = index.upsert(
    vectors=[
        ("vec1", text_emb[0].tolist()),  
        ("vec2", text_emb[1].tolist())
    ],
    namespace="example-namespace2"
)

print(index.describe_index_stats())

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'example-namespace': {'vector_count': 3},
                'example-namespace2': {'vector_count': 2}},
 'total_vector_count': 5}


In [50]:
#Querying Pinecone (the response varies depending on the searched namespace)

pinecone.init(api_key="04d65f27-278f-40f0-9cfb-c907b84115a7", environment="us-east4-gcp")
index = pinecone.Index("text-embeddings1")

query_response = index.query(
    namespace="example-namespace2",
    top_k=2,
    vector=image_emb.tolist()
)

print(query_response.matches)

[{'id': 'vec1', 'score': 0.189040869, 'values': []}, {'id': 'vec2', 'score': 0.117158741, 'values': []}]
